In [0]:

# 1) Create the storage account
# 2) create the service principal
# 3) give IAM to the servive principal


# Define the configuration for mounting
spark.conf.set("spark.databricks.acl.data.frame.enforcement", "true")

# Define your ADLS details
storage_account_name = "datafordatabricks"
container_name = "example-container"
mount_point = "/mnt/adls_mount2"

# Use a service principal for authentication
client_id = "cbe188a5-c7ac-4a04-af84-733f70d498f3"
client_secret = ""
tenant_id = "fad980a9-3948-48d2-86f5-f2a51108c262"

# Set the necessary configs
spark.conf.set(f"spark.hadoop.fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"spark.hadoop.fs.azure.account.oauth.provider.type.{storage_account_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"spark.hadoop.fs.azure.account.oauth2.client.id.{storage_account_name}.dfs.core.windows.net", client_id)
spark.conf.set(f"spark.hadoop.fs.azure.account.oauth2.client.secret.{storage_account_name}.dfs.core.windows.net", client_secret)
spark.conf.set(f"spark.hadoop.fs.azure.account.oauth2.client.endpoint.{storage_account_name}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")


configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": "cbe188a5-c7ac-4a04-af84-733f70d498f3",
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/fad980a9-3948-48d2-86f5-f2a51108c262/oauth2/token"}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
  mount_point = mount_point,
  extra_configs = configs)

print(f"Files in {mount_point}:")
files = dbutils.fs.ls(mount_point)
for file in files:
    print(f"Name: {file.name}, Path: {file.path}, Is Directory: {file.isDir}")


In [0]:
display(dbutils.fs.ls(mount_point))

In [0]:
import dlt
from pyspark.sql.functions import col

# Step 1: Define the raw data source
@dlt.table(
    comment="This table reads raw data from the mounted ADLS Gen2 path."
)
def raw_data():
    return (
        spark.read.format("csv")  # Change to the appropriate file format (e.g., parquet, json).
        .option("header", True)
        .load(mount_point)  # Replace with your specific folder in the mount.
    )

In [0]:
mnt_path = "/mnt/"

# List the files and directories in the mounted path
files = dbutils.fs.ls(mnt_path)

# Print the names of the files and directories
for file in files:
    print(f"Name: {file.name}, Path: {file.path}, Is Directory: {file.isDir}")